In [ ]:
import pandas as pd

## Our PDF reader from deck AprilHealth, masked the page with CPT code.

In [1]:
print("Behavioral Health Support for Primary Care Integrated, Virtual Mental Health Care for Family Medicine, Pediatrics and OBGYN OUR  MISSION Increasing Access to Mental Health Services in Underresourced Areas  While about a quarter of adults have a mental illness, 50 %  of the US population lives in a mental health shortage area.  [company name] sets out to help patients in areas where support is otherwise unavailable.  THE PROBLEM 25% 80% 50%  of PCP visits involve mental health Patients overwhelmingly bring their mental health concerns to primary care providers instead of going directly to specialty care. PCPs act as the de facto triage point but have limited support.of patients with mental health concerns seek support from primary care PCPs work in 15-minute appointments and typically have limited resources and training, putting a strain both on them and their patients.Of the US population lives in a mental health shortage area Given massive shortages, there are frequently 6 +  month waitlists for patients who need specialty mental health support.Communities Lack Access to Mental Healthcare Patients look to primary care as their front line of support, however providers are poorly equipped Collaborative Care ([name]) [name]  integrates physical and mental health care in the primary care setting through counseling and consulting psychiatry.  There have been 80 +  academic studies *  showing :  [name] is highly effective (2x vs. usual care) 1. [name] reduces cost of care substantially 2. [name] drives up patient and provider satisfaction 3. *See details here THE CARE MODEL [company name] Has Aligned Incentives With Primary Care Clinics Improve Outcomes  & Patient Satisfaction Studies of the [name] Model have shown that it is more than 2x as effective as usual care in driving mental health symptoms to remission.  Moreover, 75 %  of patients who are treated through [name] models are highly satisfied with their care. Activate A New  FFS Revenue Stream Collaborative care can generate significant new revenue for partner practices through FFS reimbursement. [company name] generates ~$ 10k in contrbution margin per engaged PCP / year. Improve Risk Adjustments & Reduce Cost of Care We can drive an additional increase in revenue through refining mental health diagnoses and improving risk adjustments. Additionally, [name] has been proven to reduce patient cost of care by $ 3,400 over 4 years. VALUE PROPOSITION. Thank you For more info, contact : [email]")

Behavioral Health Support for Primary Care Integrated, Virtual Mental Health Care for Family Medicine, Pediatrics and OBGYN OUR  MISSION Increasing Access to Mental Health Services in Underresourced Areas  While about a quarter of adults have a mental illness, 50 %  of the US population lives in a mental health shortage area.  [company name] sets out to help patients in areas where support is otherwise unavailable.  THE PROBLEM 25% 80% 50%  of PCP visits involve mental health Patients overwhelmingly bring their mental health concerns to primary care providers instead of going directly to specialty care. PCPs act as the de facto triage point but have limited support.of patients with mental health concerns seek support from primary care PCPs work in 15-minute appointments and typically have limited resources and training, putting a strain both on them and their patients.Of the US population lives in a mental health shortage area Given massive shortages, there are frequently 6 +  month wa

## Input the text into ChatGPT, for now in GPT 4o, summarize the health related information.

Prompt:

Here is the information on a company's product deck. Please summarize the information in three sentences, and each sentence should follow the following format: 
Sentence 1: A concise statement of the main procedure or service (e.g., type of surgery, diagnostic procedure, therapeutic intervention).
Sentence 2: A brief description of the key components or methods involved in the procedure (e.g., tools or techniques used, specific anatomy targeted).
Sentence 3 (optional): Mention of any special considerations, such as patient condition, specific approach, or adjunctive procedures that may differentiate the service from similar ones.


ChatGPT summarization: 

[Company name] provides integrated, virtual mental health care for primary care settings, including family medicine, pediatrics, and OBGYN, to increase access to mental health services in underresourced areas. The service includes counseling and consulting psychiatry, integrating mental and physical health care directly within primary care practices, leveraging collaborative care models proven to be twice as effective as usual care. This approach not only enhances patient and provider satisfaction but also generates new revenue streams for partner practices and reduces overall patient costs.

In [ ]:
def add_tokens_to_cpt_csv(csv_file, output_file, tokenizer):
    # Read the CSV into a pandas DataFrame
    df = pd.read_excel(csv_file)
    
    # Assume the CPT code description is in a column named 'description'
    df['tokens'] = df['description'].apply(lambda x: tokenizer(x, return_tensors="pt")['input_ids'][0])
    
    # Save the modified DataFrame back to a new CSV
    df.to_csv(output_file, index=False)
    print(f"Tokens added and saved to {output_file}")



In [ ]:
def seq_to_context_vec(input_text, model, tokenizer):
    # assumes bert model
    inputs = tokenizer(input_text, return_tensors='pt', padding=True, truncation=True)
    outputs = model(**inputs)
    last_hidden_state = outputs.last_hidden_state
    context_vec = last_hidden_state[:, 0, :]  # Using [CLS] token embedding for pooling
    return context_vec


In [ ]:
def get_top_k_cpt_matches(deck_summary, cpt_csv_file, model, tokenizer, k=5):
    # Read the CPT code CSV
    df = pd.read_csv(cpt_csv_file)
    
    # Tokenize the deck summary
    deck_tokens = tokenizer(deck_summary, return_tensors="pt")['input_ids'][0]
    
    # Get the contextual embeddings for the deck summary
    deck_embedding = seq_to_context_vec(deck_summary, model, tokenizer)

    # Calculate cosine similarity between deck tokens and each CPT code's tokens
    cos_sim = []
    for index, row in df.iterrows():
        cpt_description_tokens = torch.tensor(eval(row['tokens']))  # Convert token string back to tensor
        cpt_embedding = seq_to_context_vec(row['description'], model, tokenizer)  # Get CPT description embeddings
        similarity = F.cosine_similarity(deck_embedding, cpt_embedding).item()  # Calculate cosine similarity
        cos_sim.append(similarity)
    
    # Add cosine similarities to DataFrame
    df['cos_sim'] = cos_sim

    # Sort by similarity and return top K results
    top_k_df = df.sort_values(by='cos_sim', ascending=False).head(k)
    
    # Select relevant columns for output
    result = top_k_df[['CPT', 'description', 'cos_sim']].to_dict(orient='records')
    return result


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')  # Use appropriate tokenizer
add_tokens_to_cpt_csv('cpt_codes.csv', 'cpt_codes_with_tokens.csv', tokenizer)

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased')  # Use appropriate model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

deck_summary = "[Company name] provides integrated, virtual mental health care for primary care settings, including family medicine, pediatrics, and OBGYN, to increase access to mental health services in underresourced areas. The service includes counseling and consulting psychiatry, integrating mental and physical health care directly within primary care practices, leveraging collaborative care models proven to be twice as effective as usual care. This approach not only enhances patient and provider satisfaction but also generates new revenue streams for partner practices and reduces overall patient costs."
top_k_results = get_top_k_cpt_matches(deck_summary, 'cpt_codes_with_tokens.csv', model, tokenizer, k=5)

# Display the top k matches
for result in top_k_results:
    print(f"CPT Code: {result['CPT']}, Description: {result['description']}, Similarity: {result['cos_sim']}")
